In [35]:
import pandas as pd
import numpy as np
import boto3
from sagemaker import get_execution_role
import datetime

In [42]:
role = get_execution_role()
bucket='bike-demand-forecast'
data_key = 'london_weather.csv'
data_location_1 = 's3://{}/{}'.format(bucket, 'london_weather.csv')
data_location_2 = 's3://{}/{}'.format(bucket, 'santander_daily_bike_demand_2010_202207.csv')

weather_data = pd.read_csv(data_location_1)
bike_demand = pd.read_csv(data_location_2)

In [43]:
bike_demand

,Day,Number of Bicycle Hires
0,30/7/2010,"6,897"
1,31/7/2010,"5,564"
2,1/8/2010,"4,303"
3,2/8/2010,"6,642"
4,3/8/2010,"7,966"
...,...,...
4380,27/7/2022,"39,282"
4381,28/7/2022,"43,096"
4382,29/7/2022,"41,584"
4383,30/7/2022,"42,718"


In [44]:
# tidying the raw input
bike_demand['daily_bicycle_hires'] = bike_demand['Number of Bicycle Hires'].apply(lambda x: int(x.replace(",","")))
bike_demand['day_of_date'] = bike_demand['Day'].apply(lambda x : int(x[:2] if x[2]=='/' else int(x[0])))
bike_demand['Day_2'] = bike_demand.apply(lambda x : x.Day[2:] if x.day_of_date < 10 else x.Day[3:] , axis = 1)
bike_demand['mth_of_date'] = bike_demand['Day_2'].apply(lambda x : int(x[:2] if x[2]=='/' else int(x[0])))
bike_demand['yr_of_date'] = bike_demand['Day_2'].apply(lambda x : int(x[-4:]))
bike_demand['date'] = bike_demand.apply(lambda x: datetime.datetime(x.yr_of_date,x.mth_of_date,x.day_of_date),axis=1)

bike_demand = bike_demand[['date','daily_bicycle_hires']]


In [45]:
bike_demand

,date,daily_bicycle_hires
0,2010-07-30,6897
1,2010-07-31,5564
2,2010-08-01,4303
3,2010-08-02,6642
4,2010-08-03,7966
...,...,...
4380,2022-07-27,39282
4381,2022-07-28,43096
4382,2022-07-29,41584
4383,2022-07-30,42718
